# Wk18-Lecture01-CodeAlong: Preparing Time Series Data

## Learning Objectives

- By the end of this CodeAlong, students will be able to:
    - Create date time indices
    - Resample at various frequencies
    - Impute null values for time series 
    - Convert wide-form data to long-form
    

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticks

import missingno as miss

import seaborn as sns
import numpy as np
# import statsmodels.tsa.api as tsa

pd.set_option('display.float_format',lambda x:f"{x:,.2f}")

In [ ]:
sns.set_context('notebook',font_scale=0.9)
plt.style.use(['ggplot'])

# Datetime Objects

In Python, we will deal with 2 types of date/time objects:
- Python datetime
- Panda's datetime indices

In [ ]:
import datetime as dt

In [ ]:
# Get Today's date (ONLY) from python's datetime.date module
today_date = dt.date.today()
today_date

In [ ]:
# Printed dt objects look very different!
print(today_date)

In [ ]:
# Get Today's date(&time) from python's datetime.datetime module
today_datetime = dt.datetime.today()
today_datetime

In [ ]:
print(today_datetime)

### Datetime Formats

- Datetime objects have a `.strftime()` method (string-format-time)

- 📖 **strftime cheat sheet: https://strftime.org/**
- 📖 **Official Table of Python datetime format codes: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes **


| Directive   | Meaning                                                                                                                                                                          | Example                                                                      |
|:------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------|
| %a          | Weekday as locale’s abbreviated name.                                                                                                                                            | Sun, Mon, …, Sat (en_US); So, Mo, …, Sa (de_DE)                              |
| %A          | Weekday as locale’s full name.                                                                                                                                                   | Sunday, Monday, …, Saturday (en_US); Sonntag, Montag, …, Samstag (de_DE)     |
| %w          | Weekday as a decimal number, where 0 is Sunday and 6 is Saturday.                                                                                                                | 0, 1, …, 6                                                                   |
| %d          | Day of the month as a zero-padded decimal number.                                                                                                                                | 01, 02, …, 31                                                                |
| %b          | Month as locale’s abbreviated name.                                                                                                                                              | Jan, Feb, …, Dec (en_US); Jan, Feb, …, Dez (de_DE)                           |
| %B          | Month as locale’s full name.                                                                                                                                                     | January, February, …, December (en_US); Januar, Februar, …, Dezember (de_DE) |
| %m          | Month as a zero-padded decimal number.                                                                                                                                           | 01, 02, …, 12                                                                |
| %y          | Year without century as a zero-padded decimal number.                                                                                                                            | 00, 01, …, 99                                                                |
| %Y          | Year with century as a decimal number.                                                                                                                                           | 0001, 0002, …, 2013, 2014, …, 9998, 9999                                     |
| %H          | Hour (24-hour clock) as a zero-padded decimal number.                                                                                                                            | 00, 01, …, 23                                                                |
| %I          | Hour (12-hour clock) as a zero-padded decimal number.                                                                                                                            | 01, 02, …, 12                                                                |
| %p          | Locale’s equivalent of either AM or PM.                                                                                                                                          | AM, PM (en_US); am, pm (de_DE)                                               |
| %M          | Minute as a zero-padded decimal number.                                                                                                                                          | 00, 01, …, 59                                                                |
| %S          | Second as a zero-padded decimal number.                                                                                                                                          | 00, 01, …, 59                                                                |
| %f          | Microsecond as a decimal number, zero-padded to 6 digits.                                                                                                                        | 000000, 000001, …, 999999                                                    |
| %z          | UTC offset in the form ±HHMM[SS[.ffffff]] (empty string if the object is naive).                                                                                                 | (empty), +0000, -0400, +1030, +063415, -030712.345216                        |
| %Z          | Time zone name (empty string if the object is naive).                                                                                                                            | (empty), UTC, GMT                                                            |
| %j          | Day of the year as a zero-padded decimal number.                                                                                                                                 | 001, 002, …, 366                                                             |
| %U          | Week number of the year (Sunday as the first day of the week) as a zero-padded decimal number. All days in a new year preceding the first Sunday are considered to be in week 0. | 00, 01, …, 53                                                                |
| %W          | Week number of the year (Monday as the first day of the week) as a zero-padded decimal number. All days in a new year preceding the first Monday are considered to be in week 0. | 00, 01, …, 53                                                                |
| %c          | Locale’s appropriate date and time representation.                                                                                                                               | Tue Aug 16 21:30:00 1988 (en_US); Di 16 Aug 21:30:00 1988 (de_DE)            |
| %x          | Locale’s appropriate date representation.                                                                                                                                        | 08/16/88 (None); 08/16/1988 (en_US); 16.08.1988 (de_DE)                      |
| %X          | Locale’s appropriate time representation.                                                                                                                                        | 21:30:00 (en_US); 21:30:00 (de_DE)                                           |
| %%          | A literal '%' character.                                                                                                                                                         | %                                                                            |

In [ ]:
## How to make a markdown table from a csv
# freq_codes = pd.read_csv('Reference-Python-Frequency-Codes.csv', index_col=0)
# print(freq_codes.drop(columns=["Notes"]).to_markdown())

In [ ]:
## Month/Day/Year 24Hour Time
mo_day_yr_24hr_time_fmt = "%m/%d/%Y %T"
today_dt_24hr_str = today_datetime.strftime(mo_day_yr_24hr_time_fmt)
today_dt_24hr_str

In [ ]:
## Monday/Day/Year 12 Hour Time (+AM/PM)
mo_day_yr_12hr_time_fmt =  "%m/%d/%Y %I:%M:%S %p"

today_dt_12hr_str = today_datetime.strftime(mo_day_yr_12hr_time_fmt)
today_dt_12hr_str

> `.strftime` returns a STRING

In [ ]:
type(today_dt_24hr_str)

In [ ]:
## But pd.to_datetime can do that for us! 
converted_date_pandas = pd.to_datetime(today_dt_24hr_str)
converted_date_pandas

> We will use pandas datetime objects/functions whenever possible, as they save us a lot of time!

### Comparing Python vs. Pandas Datetime Objects

In [ ]:
## Python Datetime 
print(type(today_datetime))

## Displaying vs printing
display(today_datetime)
print(today_datetime)

In [ ]:
## Panda's Timestamp
print(type(converted_date_pandas))

## Displaying vs printing
display(converted_date_pandas)
print(converted_date_pandas)

# Working with Time Series - Data with Regular Intervals

### Stock Data (S&P500)

- S&P 500 (Kibot) 
     - IVE S&P 500 1-min aggregate data from: http://www.kibot.com/free_historical_data.aspx

In [ ]:
sp500_min_df = pd.read_csv('Data/stocks/sp500_ive_min_tick_bidask.csv')
sp500_min_df.head()

## Using pd.to_datetime

- Powerful  (but slow) function for converting data to datetime.

In [ ]:
%%time
## Make a datetime index using pd.to_datetime
sp500_min_df['Datetime'] = pd.to_datetime(sp500_min_df['Date']+' ' + sp500_min_df['Time'])
sp500_min_df

- We can speed up the conversion by specifying the correct time format and setting `infer_datetime_format=False`

In [ ]:
## Creating/testing our time format
fmt = "%m/%d/%Y %H:%M"
today_datetime.strftime(fmt)

In [ ]:
%%time
## Make a datetime index using pd.to_datetime & specifying the format  (to save gtime)
sp500_min_df['Datetime'] = pd.to_datetime(sp500_min_df['Date']+' ' + sp500_min_df['Time'],
                                          format=fmt,
                                         infer_datetime_format=False)
sp500_min_df

In [ ]:
## Setting the datetime index
df_ts = sp500_min_df.set_index("Datetime")
df_ts

In [ ]:
df_ts.index

In [ ]:
## Only keeping Closing Ask/Bid
close_cols = [c for c in df_ts.columns if "Close" in c]
df_ts = df_ts[close_cols]
df_ts

In [ ]:
# check the index to confirm its datetime
df_ts.index

## 📈 Visualizing Our Time Series DataFrame

In [ ]:
## let's visualize our dataframe 
ax = df_ts.plot()

> Time series visualizations usually require wider figures. 

### Changing Default Figure Size

In [ ]:
# check default figsize in rcParams
plt.rcParams['figure.figsize']

In [ ]:
## Change the default figure size to something wider (12,5)
plt.rcParams['figure.figsize'] = (12,5)

In [ ]:
## recreate the plot
ax = df_ts.plot()

### Using Tick Date Formatters/Locators

- Let's add a minor xtick every 3 months.

In [ ]:
## CREATE ARTISTS FOR MAJOR XTICKS (Years)
# Create a year locator
loc_major_yr = mdates.YearLocator()
# Create a year formatter using 4-digit years
fmt_major_yr = mdates.DateFormatter("%Y")


## CREATE ARTISTS FOR MINOR XTICKS ( Months)
# Create a month locator that will add months at 1,4,7,10
loc_minor_3m = mdates.MonthLocator(bymonth=[1,4,7,10])
# Createa monthnformatter that will use 3-letter month names
fmt_minor_3m = mdates.DateFormatter("%b")

In [ ]:
## Create our plot and save the ax
ax = df_ts.plot()
ax.set(ylabel="Stock Value", title='S&P 500')

# Set xaxis major locator/formatter
ax.xaxis.set_major_locator(loc_major_yr)
ax.xaxis.set_major_formatter(fmt_major_yr)


# Set xaxis minor locator/formatter
ax.xaxis.set_minor_locator(loc_minor_3m)
ax.xaxis.set_minor_formatter(fmt_minor_3m)

# Add gridlines for major xaxis ticks
ax.grid(which='major',axis='x',color='k',ls=':',lw=1)

## Rotate the major tick years using fig.autofmt_xdate
fig = ax.get_figure()
fig.autofmt_xdate(which='major', rotation=90,ha='center')

In [ ]:
def format_xdates_stocks(ax):
    
    # Create a year locator
    loc_major_yr = mdates.YearLocator()
    # Create a year formatter using 4-digit years
    fmt_major_yr = mdates.DateFormatter("%Y")


    # Create a month locator that will add months at 1,4,7,10
    loc_minor_3m = mdates.MonthLocator(bymonth=[1,4,7,10])
    # Createa monthnformatter that will use 3-letter month names
    fmt_minor_3m = mdates.DateFormatter("%b")
    

    
    # Set xaxis major locator/formatter
    ax.xaxis.set_major_locator(loc_major_yr)
    ax.xaxis.set_major_formatter(fmt_major_yr)


    # Set xaxis minor locator/formatter
    ax.xaxis.set_minor_locator(loc_minor_3m)
    ax.xaxis.set_minor_formatter(fmt_minor_3m)

    

    # Add gridlines for major xaxis ticks
    ax.grid(which='major',axis='x',color='k',ls=':',lw=1)

    ## Rotate the major tick years using fig.autofmt_xdate
    fig = ax.get_figure()
    fig.autofmt_xdate(which='major', rotation=90,ha='center')
    return fig


In [ ]:
ax = df_ts.plot()
format_xdates_stocks(ax);

## Slicing (with a Datetime Index)

In [ ]:
## Get all/any dates from 2022
df_ts.loc['2022']

In [ ]:
# Get all of january, 2022
df_ts.loc['January 2022']

In [ ]:
## Get January 01st, 2022 to March 16th 2022
df_ts.loc['01/01/22':'03/16/22']

> Notice the earliest date is 2020-01-03

In [ ]:
## Get just 01/03 - AskClose
df_ts.loc["01/03/2022", "AskClose"]

In [ ]:
## cannot slice out more than 1 non-continuous dates
dates_to_keep = ['01/03/2022','01/05/2022']
try: 
    df_ts.loc[dates_to_keep]
except Exception as e:
    display(e)

In [ ]:
## But can use multiple .loc's in a loop or list comprehension
sliced_ts = pd.concat([df_ts.loc[i] for i in dates_to_keep])
sliced_ts

In [ ]:
## saving ts to return to later
stock_ts = df_ts.loc['2020':'2020','AskClose'].copy()
stock_ts.plot()

### Using Time Deltas

> We want to slice out a ts around the date with the minumum price. We want our slice to start 5 days before and end 5 days after.

In [ ]:
## save timestamp of lowest price
idx_min_price = stock_ts.idxmin()
idx_min_price

In [ ]:
try:
    idx_min_price +'4 days'
except Exception as e:
    display(e)

In [ ]:
## Making a time delta with pd.to_tiemdelta
delta_days = pd.to_timedelta('4 days')
delta_days

In [ ]:
## Subtracting our delta to get start
start = idx_min_price - delta_days
start

In [ ]:
## adding our time delta to get the end
end = idx_min_price + delta_days
end

In [ ]:
## Annotating the time period we've selected on our entire ts
ax = stock_ts.plot(title="Annotating Start and End Dates Calcualted with Timedeltas");
ax.axvline(start,ls='--',color='green', label='Start Date')
ax.axvline(end, ls='--',color='red',label='End Date')
ax.legend()

In [ ]:
## Slicing out the timeperiod and saving as eda_ts
eda_ts = stock_ts.loc[start:end]#.resample("D").last()
eda_ts

In [ ]:
ax = eda_ts.plot()

In [ ]:
eda_ts.plot(style='.--');

In [ ]:
ax = eda_ts.plot(style='.');
ax.axvspan('2020-03-21','2020-03-23',alpha=0.5)

> Why does it look the way it does?? Are there null values?

In [ ]:
## There are no null values in the ts var...
eda_ts.isna().sum()

> What day of the week were the days missing data?

In [ ]:
## what day of the week were the days missing data?
pd.to_datetime('2020-03-21').day_name()

In [ ]:
pd.to_datetime('2020-03-22').day_name()

In [ ]:
# what is our datetime index's frequency?
eda_ts.index

In [ ]:
ts_min = eda_ts.resample("T").asfreq()
ts_min

## Imputing Null Values (time series)

In [ ]:

ax = ts_min.fillna(ts_min.mean()).plot(style='.-')
ax.axvspan('2020-03-21','2020-03-23',alpha=0.5)
ax.set(title="mean", ylabel="Stock Price");

In [ ]:
ax = ts_min.fillna(method='bfill').plot(style='.-')
ax.axvspan('2020-03-21','2020-03-23',alpha=0.5)
ax.set(title='bfill', ylabel="Stock Price");

In [ ]:
ax = ts_min.fillna(method='ffill').plot(style='.-')
ax.axvspan('2020-03-21','2020-03-23',alpha=0.5)
ax.set(title='ffill', ylabel="Stock Price");

In [ ]:
ts_temp = ts_min.interpolate()
ax = ts_temp.fillna(method='ffill').plot(style='.-')
ax.axvspan('2020-03-21','2020-03-23',alpha=0.5)
ax.set(title='ffill', ylabel="Stock Price");

# Working with Wide-Form Time Series Data

In [ ]:
crypto = pd.read_csv("Data/stocks/wide-form-crypto.csv")
crypto

In [ ]:
## First 3 columns are id coluimns
id_vars = crypto.columns[:3]
id_vars

In [ ]:
## Melt the crypto data 
long_crypto = pd.melt(crypto,id_vars=id_vars)
long_crypto

In [ ]:
## Set the var_name  as "Date" and the value_name was "Currency Value"
long_crypto = pd.melt(crypto,id_vars=id_vars, var_name='Date', 
                      value_name="Value")
long_crypto

In [ ]:
## Now make Date column datetime 
long_crypto['Date'] = pd.to_datetime( long_crypto["Date"])
long_crypto

In [ ]:
## Set Date as index
long_crypto = long_crypto.set_index('Date')
long_crypto

In [ ]:
## 
long_crypto.index

In [ ]:
long_crypto.groupby('CryptoCurrency').resample("D").asfreq()

In [ ]:
crypto_ts = long_crypto.groupby('CryptoCurrency').resample("D").asfreq()['Value']
crypto_ts

In [ ]:
crypto_ts.plot();

> What are we seeing?

In [ ]:
crypto_ts.unstack(0)

In [ ]:
crypto_unstacked = crypto_ts.unstack(0)
crypto_unstacked.plot();

## Selecting our TS for Resampling Demonstration

In [ ]:
ts = crypto_unstacked['Bitcoin'].loc['2021':'2022'].copy()
ts

In [ ]:
ts.plot();

## Resampling

In [ ]:
## Check the index to confirm the current freq
ts.index

>It looks like we have minute-resolution data (frequency = minutes).

### Resample as Weeks Frequency

- We need to look up the correct frequency code for minutes: https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects 
    

In [ ]:
## Resample using correct freq code and use .asfreq as agg emthod
ts_W = ts.resample("W").last()
ts_W

In [ ]:
ts_W.index

In [ ]:
ts.plot()

- Let's resample our ts as daily, weekly, and monthly to compare.

In [ ]:
freqs = ['D','W','M','Q','A']


for freq in freqs:
    ax = ts.plot(label='Original',  style='.-',
            title='Comparing Resampled Frequencies');

    ts_temp = ts.resample(freq).last()
    ts_temp.plot(style='o-', label=freq,ax=ax)
    plt.legend()
    plt.show()